# Track width a wheelbase study
### Dave Yonkers, June 2020

## Aero:

In [55]:
import numpy as np
import pandas as pd

# data from James' sims
data = [[45, 223,   90,   48,  8.6,   78, 29.5,    89, 35.5, 0.33],
        [46, 233, 93.6, 50.3,    9, 94.4, 30.5,    96,   35, 0.32],
        [47, 229, 92.5, 49.8,  8.8,   93, 28.7,  95.5, 38.5, 0.30],
        [48, 257, 95.5, 56.8,  9.9,  108,   30,  99.3, 40.3, 0.36],
        [49, 268, 95.8, 59.5, 10.1,  114, 28.4,   102,   41, 0.37],
        [50, 285,  100, 65.8, 10.9,  120,   30, 108.5, 43.5, 0.37]]
aero_df = pd.DataFrame(data)
aero_df.columns = ["track_width", "downforce", "drag", "fw_downforce", "fw_drag", "tray_downforce", "tray_drag", "rw_downforce", "rw_drag", "abal"]
aero_df.set_index("track_width", inplace=True)

# show the dataframe below
aero_df

,downforce,drag,fw_downforce,fw_drag,tray_downforce,tray_drag,rw_downforce,rw_drag,abal
track_width,,,,,,,,,
45,223,90.0,48.0,8.6,78.0,29.5,89.0,35.5,0.33
46,233,93.6,50.3,9.0,94.4,30.5,96.0,35.0,0.32
47,229,92.5,49.8,8.8,93.0,28.7,95.5,38.5,0.30
48,257,95.5,56.8,9.9,108.0,30.0,99.3,40.3,0.36
49,268,95.8,59.5,10.1,114.0,28.4,102.0,41.0,0.37
50,285,100.0,65.8,10.9,120.0,30.0,108.5,43.5,0.37


### Create a regression
only use 45 & 50 inch track widths, since we know it should "in theory" scale linearly and those are the most accurate sims.

In [56]:
# generate a polynomial and a function to predict y-values
coefs = np.polyfit([45, 50], [aero_df["downforce"][45], aero_df["downforce"][50]], deg=1)
downforce_function = np.poly1d(coefs)

# now for drag
coefs = np.polyfit([45, 50], [aero_df["drag"][45], aero_df["drag"][50]], deg=1)
drag_function = np.poly1d(coefs)

aero_df["pred_downforce"] = downforce_function(aero_df.index)
aero_df["pred_drag"] = drag_function(aero_df.index)

In [57]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# create subplots objects
fig = make_subplots(rows=1, cols=2, subplot_titles=("Downforce vs Track Width", "Drag vs Track Width"))

# create the scatter traces to add
downforce_trace = go.Scattergl(x=aero_df.index, y=aero_df["downforce"], mode='markers', name="CFD downforce")
drag_trace = go.Scattergl(x=aero_df.index, y=aero_df["drag"], mode='markers', name="CFD drag")

# create the line traces to add
pred_downforce_trace = go.Scattergl(x=aero_df.index, y=aero_df["pred_downforce"], mode='lines', name="downforce regression")
pred_drag_trace = go.Scattergl(x=aero_df.index, y=aero_df["pred_drag"], mode='lines', name="drag regression")

# add the traces to the subplots
fig.add_trace(downforce_trace, row=1, col=1)
fig.add_trace(drag_trace, row=1, col=2)
fig.add_trace(pred_downforce_trace, row=1, col=1)
fig.add_trace(pred_drag_trace, row=1, col=2)

# additional formatting
fig.update_xaxes(title_text="Track Width", row=1, col=1)
fig.update_xaxes(title_text="Track Width", row=1, col=2)
fig.update_yaxes(title_text="Downforce (lbs)", row=1, col=1)
fig.update_yaxes(title_text="Drag (lbs)", row=1, col=2)

fig.show()

In [58]:
# create scale factors and a regression to generate them along the lines shown above
aero_df["downforce_scale"] = aero_df["downforce"] / aero_df["downforce"][50]
aero_df["drag_scale"]      = aero_df["drag"]      / aero_df["drag"][50]

# generate regressions for the 49 & 50 inch TWs
coefs = np.polyfit([45, 50], [aero_df["downforce_scale"][45], aero_df["downforce_scale"][50]], deg=1)
downforce_scale_factor = np.poly1d(coefs)

# now for drag
coefs = np.polyfit([45, 50], [aero_df["drag_scale"][45], aero_df["drag_scale"][50]], deg=1)
drag_scale_factor = np.poly1d(coefs)

aero_df

,downforce,drag,fw_downforce,fw_drag,tray_downforce,tray_drag,rw_downforce,rw_drag,abal,pred_downforce,pred_drag,downforce_scale,drag_scale
track_width,,,,,,,,,,,,,
45,223,90.0,48.0,8.6,78.0,29.5,89.0,35.5,0.33,223.0,90.0,0.782456,0.900
46,233,93.6,50.3,9.0,94.4,30.5,96.0,35.0,0.32,235.4,92.0,0.817544,0.936
47,229,92.5,49.8,8.8,93.0,28.7,95.5,38.5,0.30,247.8,94.0,0.803509,0.925
48,257,95.5,56.8,9.9,108.0,30.0,99.3,40.3,0.36,260.2,96.0,0.901754,0.955
49,268,95.8,59.5,10.1,114.0,28.4,102.0,41.0,0.37,272.6,98.0,0.940351,0.958
50,285,100.0,65.8,10.9,120.0,30.0,108.5,43.5,0.37,285.0,100.0,1.000000,1.000


## Aero weight

In [59]:
# stuff that miko calculated for track width
tw = [50, 49, 48, 47, 46, 45]
weight = [39.087, 38.721, 38, 37.423, 36.985, 36.551]

# calculate a linear regression and take the rate of change
coefs = np.polyfit(tw, weight, deg=1)
tw_aero_lbs_per_inch = coefs[0]

## Suspension weight

This number is the weight/lateral-inch of the suspension a-arms and tie-rods. If the track width increases/decreases, this is where the bulk of the extra weight will be in terms of suspension components

In [60]:
tw_sus_lbs_per_inch = 2 * .08 + 2 * 0.076  # 2*front + 2*rear

## Chassis weight
The increased weight from wheelbase will come primarily from an increase of chassis length

In [61]:
wb_chassis_lbs_per_inch = 45 / 80  # 45lbs straight out of the autoclave / 80in 

## Creating the runs array

In [62]:
min_tw = 40
max_tw = 55
step_tw = 1
tws = np.arange(min_tw, max_tw + step_tw, step_tw)

min_wb = 60.5
max_wb = 80.5
step_wb = 1
wbs = np.arange(min_wb, max_wb + step_wb, step_wb)

num_tracks = 7  # indexed 1-7 b/c matlab...
# 1 = accel
# 2 = skidpad
# 3 = autocross
# 4 = slalom_min
# 5 = slalom_med
# 6 = slalom_max
# 7 = all-in-one
# 8 = endurance - not running at the moment

baseline_vehicle_weight = 599  # lbs
baseline_vehicle_tw = 50
baseline_vehicle_wb = 60.5

# weight distobution % front
weight_dists = [0.30, 0.35, 0.40, 0.45, 0.5, 0.55, 0.60, 0.65, 0.70]

# empty runs array
runs = []

for track in range(1, num_tracks + 1):
    for tw in tws:
        for wb in wbs:
            for wd in weight_dists:
            
                # calculate the aero scale factors
                downforce_scaler = downforce_scale_factor(tw)
                drag_scaler = drag_scale_factor(tw)


                # calcuate weight differences
                tw_diff = tw - baseline_vehicle_tw
                wb_diff = wb - baseline_vehicle_wb

                weight_diff = 0
                weight_diff += tw_diff * tw_sus_lbs_per_inch
                weight_diff += tw_diff * tw_aero_lbs_per_inch
                weight_diff += wb_diff * wb_chassis_lbs_per_inch
                vehicle_weight = baseline_vehicle_weight + weight_diff
                
                # weight distrobution stuff
                front_back_weight_split = vehicle_weight / 2
                fr_weight = front_back_weight_split * wd
                fl_weight = front_back_weight_split * wd
                rr_weight = front_back_weight_split * (1.0 - wd)
                rl_weight = front_back_weight_split * (1.0 - wd)

                # create the run array
                run_array = (track, tw, wb, downforce_scaler, drag_scaler, fr_weight, fl_weight, rr_weight, rl_weight)
                runs.append(run_array)

# turn the runs list into a dataframe
runs_df = pd.DataFrame(runs)
runs_df.columns = ("track_num", "tw", "wb", "df_scale", "drag_scale", "fr_weight", "fl_weight", "rr_weight", "rl_weight")
runs_df

,track_num,tw,wb,df_scale,drag_scale,fr_weight,fl_weight,rr_weight,rl_weight
0,1,40,60.5,0.564912,0.8,88.590643,88.590643,206.711500,206.711500
1,1,40,60.5,0.564912,0.8,103.355750,103.355750,191.946393,191.946393
2,1,40,60.5,0.564912,0.8,118.120857,118.120857,177.181286,177.181286
3,1,40,60.5,0.564912,0.8,132.885964,132.885964,162.416179,162.416179
4,1,40,60.5,0.564912,0.8,147.651071,147.651071,147.651071,147.651071
...,...,...,...,...,...,...,...,...,...
21163,7,55,80.5,1.217544,1.1,153.611964,153.611964,153.611964,153.611964
21164,7,55,80.5,1.217544,1.1,168.973161,168.973161,138.250768,138.250768
21165,7,55,80.5,1.217544,1.1,184.334357,184.334357,122.889571,122.889571
21166,7,55,80.5,1.217544,1.1,199.695554,199.695554,107.528375,107.528375


In [63]:
# save the runs array as a csv!
runs_df.to_csv("runs_insight.csv")

# Simulation Analysis

In [110]:
# the file path to the results csv file (may be different on your system)
results_file_path = "/Users/daveyonkers/Michigan State University/RSO-MSU Formula Racing Team - General/SR22 - Car XX/Systems/Simulation/working_dir/DOE/track_width_wheel_base/tw_wb_RESULTS.csv"

# load the results into a dataframe
results_df= pd.read_csv(results_file_path)

results_df.head()

,track_num,tw,wb,df_scale,drag_scale,fr_weight,fl_weight,rr_weight,rl_weight,!laptime,...,slipaFR,slipaRL,slipaRR,sliprFL,sliprFR,sliprRL,sliprRR,speed,yawa,yawv
0,1,40,60.5,0.564912,0.8,88.590643,88.590643,206.711500,206.711500,4.086,...,0.12721,0.13786,-0.009197,0.014902,0.014922,0.050309,0.050421,63.142,2.4664,0.000087
1,1,40,60.5,0.564912,0.8,103.355750,103.355750,191.946393,191.946393,4.086,...,0.12721,0.13786,-0.009197,0.014902,0.014922,0.050309,0.050421,63.142,2.4664,0.000087
2,1,40,60.5,0.564912,0.8,118.120857,118.120857,177.181286,177.181286,4.086,...,0.12721,0.13786,-0.009197,0.014902,0.014922,0.050309,0.050421,63.142,2.4664,0.000087
3,1,40,60.5,0.564912,0.8,132.885964,132.885964,162.416179,162.416179,4.086,...,0.12721,0.13786,-0.009197,0.014902,0.014922,0.050309,0.050421,63.142,2.4664,0.000087
4,1,40,60.5,0.564912,0.8,147.651071,147.651071,147.651071,147.651071,4.086,...,0.12721,0.13786,-0.009197,0.014902,0.014922,0.050309,0.050421,63.142,2.4664,0.000087


In [111]:
# Back-calculate the weight distrobution (%F)
weight_dists = results_df_no_wd["fr_weight"] / (results_df_no_wd["fr_weight"] + results_df_no_wd["rr_weight"])
# add as a column to the results df
results_df.insert(5, "weight_dst", weight_dists, True)

### How many simulations did not solve properly?

In [112]:
# the sim reports results of -999 for simulation errors
results_df[results_df["!laptime"] < 0]

,track_num,tw,wb,df_scale,drag_scale,weight_dst,fr_weight,fl_weight,rr_weight,rl_weight,...,slipaFR,slipaRL,slipaRR,sliprFL,sliprFR,sliprRL,sliprRR,speed,yawa,yawv
6048,3,40,60.5,0.564912,0.80,0.30,88.590643,88.590643,206.711500,206.711500,...,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9
6049,3,40,60.5,0.564912,0.80,0.35,103.355750,103.355750,191.946393,191.946393,...,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9
6050,3,40,60.5,0.564912,0.80,0.40,118.120857,118.120857,177.181286,177.181286,...,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9
6051,3,40,60.5,0.564912,0.80,0.45,132.885964,132.885964,162.416179,162.416179,...,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9
6052,3,40,60.5,0.564912,0.80,0.50,147.651071,147.651071,147.651071,147.651071,...,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20767,7,53,78.5,1.130526,1.06,0.50,152.910929,152.910929,152.910929,152.910929,...,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9
20768,7,53,78.5,1.130526,1.06,0.55,168.202021,168.202021,137.619836,137.619836,...,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9
20769,7,53,78.5,1.130526,1.06,0.60,183.493114,183.493114,122.328743,122.328743,...,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9
20770,7,53,78.5,1.130526,1.06,0.65,198.784207,198.784207,107.037650,107.037650,...,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9


### Replace the simulations that did not solve properly with np.nan

In [113]:
for index in results_df.index:
    for column in results_df.columns:
        # if the value at the current location is < -995, then there was almost certainly an error
        if results_df.at[index, column] <= -995:
            results_df.at[index, column] = np.nan
            
results_df

,track_num,tw,wb,df_scale,drag_scale,weight_dst,fr_weight,fl_weight,rr_weight,rl_weight,...,slipaFR,slipaRL,slipaRR,sliprFL,sliprFR,sliprRL,sliprRR,speed,yawa,yawv
0,1,40,60.5,0.564912,0.8,0.30,88.590643,88.590643,206.711500,206.711500,...,0.12721,0.13786,-0.009197,0.014902,0.014922,0.050309,0.050421,63.142,2.4664,0.000087
1,1,40,60.5,0.564912,0.8,0.35,103.355750,103.355750,191.946393,191.946393,...,0.12721,0.13786,-0.009197,0.014902,0.014922,0.050309,0.050421,63.142,2.4664,0.000087
2,1,40,60.5,0.564912,0.8,0.40,118.120857,118.120857,177.181286,177.181286,...,0.12721,0.13786,-0.009197,0.014902,0.014922,0.050309,0.050421,63.142,2.4664,0.000087
3,1,40,60.5,0.564912,0.8,0.45,132.885964,132.885964,162.416179,162.416179,...,0.12721,0.13786,-0.009197,0.014902,0.014922,0.050309,0.050421,63.142,2.4664,0.000087
4,1,40,60.5,0.564912,0.8,0.50,147.651071,147.651071,147.651071,147.651071,...,0.12721,0.13786,-0.009197,0.014902,0.014922,0.050309,0.050421,63.142,2.4664,0.000087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21163,7,55,80.5,1.217544,1.1,0.50,153.611964,153.611964,153.611964,153.611964,...,6.59930,3.65360,2.994700,0.021668,0.025350,0.064476,0.096046,72.734,363.3500,73.357000
21164,7,55,80.5,1.217544,1.1,0.55,168.973161,168.973161,138.250768,138.250768,...,6.59930,3.65360,2.994700,0.021668,0.025350,0.064476,0.096046,72.734,363.3500,73.357000
21165,7,55,80.5,1.217544,1.1,0.60,184.334357,184.334357,122.889571,122.889571,...,6.59930,3.65360,2.994700,0.021668,0.025350,0.064476,0.096046,72.734,363.3500,73.357000
21166,7,55,80.5,1.217544,1.1,0.65,199.695554,199.695554,107.528375,107.528375,...,6.59930,3.65360,2.994700,0.021668,0.025350,0.064476,0.096046,72.734,363.3500,73.357000


### Create a utility function that will allow us to easily extract TW vs WB results into 3-dimensions

In [114]:
# beware, this function uses globals! Specifically speaking, min_tw, tws, step_tw... etc. as well as the DF!
def create_tw_wb_results_grid(track_num, weight_dist_flt, results_col_str):
    """
    pulls out data into a TWxWB grid so that heatmaps may be plotted. Essentially, this pulls out whatever
    results data you input in results_col_str and turns it into "z" data with x and y being tw and wb
    :param track_num: int - track number
    :param weight_dist_flt: float - the weight distribution
    :param results_col_str: string - the title of the specific column to pul data from
    :returns: the "z" data 2d array
    """

    # create a mesh grid to place results
    tw_wb_ary = np.zeros(shape=(len(wbs), len(tws)))

    for tw in tws:
        for wb in wbs:

            # create track width and wheelbase masks that we can combine to "index" the data we want
            tw_msk = results_df["tw"] == tw
            wb_msk = results_df["wb"] == wb
            track_msk = results_df["track_num"] == track_num
            wd_msk = results_df["weight_dst"] == weight_dist_flt
            
            # generate the tw index and wb index to place in the mesh grid
            tw_i = int((tw - min_tw) / step_tw)
            wb_i = int((wb - min_wb) / step_wb)
            
            # combine all of the masks and pull out the desired result
            tw_wb_ary[wb_i][tw_i] = results_df[tw_msk & wb_msk & track_msk & wd_msk][results_col_str]
            
    return tw_wb_ary

# Acceleration

In [119]:
# create subplots objects

laptimes = create_tw_wb_results_grid(7, 0.50, "!laptime")
        
fig = go.Figure(data=go.Heatmap(
                   z=laptimes,
                   x=tws,
                   y=wbs,
                   hoverongaps = False,
                   colorscale = 'Viridis_r'))

fig.update_layout(
    title="Wheelbase vs. Track Width vs. Laptime — Acceleration (w.d. = 50%F)",
    xaxis_title="Track Width",
    yaxis_title="Wheelbase",
)
fig.show()